# Generátorok használata a RealPython leírása mentén

Eredeti cikk:  
https://realpython.com/introduction-to-python-generators/

# egyszerű generátor függvény

In [4]:


# függvényt csinálunk, ami nem return-nal ad vissza értéket, hanem yield-del
def infinite_sequence():
    num = 0
    while True:
        yield num
        num += 1

# a függvény egy "lazy iterator" objektumot ad vissza, amin értelmezve van a next() hívás
gen = infinite_sequence()

# hívogatjuk és működik...
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))


0
1
2
3
4
5


# Fájlolvasás, sorok számlálása  
paraszos módszer

In [3]:
# előkészület
# import és fájlnév megadása
from pathlib import Path
data_folder = Path('C:\\Users\\trifonov\\Dropbox\\Calibre\\Irving Stone\Michelangelo (879)')
infile = data_folder / 'Michelangelo - Irving Stone.txt'


## simán felolvassuk a fájlt egy függvénnyel

In [4]:

# felolvassuk a teljes fájlt és visszaadjuk a teljes tartalmat
def csv_reader(file_name):
    file = open(file_name, encoding='utf-8')

    # a beolvasott mindent sorokra tördelünk
    result = file.read().split("\n")

    # visszaadjuk 
    return result


# itt adódik át az egész könyv 
csv_gen = csv_reader(infile)
row_count = 0

# a visszakapott listát megszámoljuk 
for row in csv_gen:
    row_count += 1

print(f"Row count is {row_count}")

Row count is 18371


## generator comprehension  használatával  
mint a *list comprehension*, csak **[...]** helyett **(...)**

In [5]:
# itt készül a generator comprehension
# mellesleg az open() - ha az encoding megfelelő ÉS text
# módban olvasunk, akkor sorokat ad vissza
csv_gen = (row for row in open(infile, encoding='utf-8'))

for i,line in enumerate(csv_gen):
    # kihagyjuk az üres sorokat, a többit kiírjuk
    if line != '\n': print('<{}>'.format(line), end='')
    # a huszadikig
    if i>20: break

# meg még csak úgy, hogy a next() is működik
print('next: {}'.format(next(csv_gen)), end='')
print('next: {}'.format(next(csv_gen)), end='')
print('next: {}'.format(next(csv_gen)), end='')
print('next: {}'.format(next(csv_gen)), end='')
print('next: {}'.format(next(csv_gen)), end='')



<Irving Stone – Michelangelo
><Regényes életrajz
><„Dávid ​​csak egyedül állhat a talpazaton… Ha a véres fej nincs ott, Góliátnál nagyobb ellenségek legyőzésének diadalmas szimbóluma lehet… Az ő Dávidja nemcsak erős harcos, de az emberi szellem, a lélek és értelem gigásza is, nagyot mer az élet minden vonatkozásában…”
><„Megvan hozzá az esze, akarata, belső ereje, hogy egy új világot teremtsen és betöltse azt az emberi szellem alkotásaival. Az ő Dávidja Apolló lesz, de több, mint az, Héraklész lesz és Ádám, de sokkal több azoknál – az ember legtökéletesebb megvalósulása lesz, aki egy értelemmel és emberséggel áthatott világban fejti ki a maga erőit…” – És megszületett az utolérhetetlen szépségű, erőt és értelmet sugárzó szobor, hogy Firenze és a reneszánsz szimbólumává válva, századokon át hirdesse egy izgalmas, véres küzdelmekben és hatalmas szellem: tettekben egyaránt bővelkedő kor eszményeinek s egy nagy alkotó teremtő erejének csodálatos találkozását.
><Stone hatalmas felkészültség

## a next() függvény
tulajdonképpen csak a __next__() metódust hívja.  
csv_gen.__next__() ugyanat mint next(csv_gen)


In [34]:
csv_gen.__next__()


'\n'

In [73]:


def is_palindrome(num):

    if num % 10 == 0: return False

    temp = num
    reversed = 0
    while temp != 0:
        reversed = (10 * reversed) + (temp % 10)
        temp = temp // 10

    if num == reversed:
        return True
    else:
        return False

# generator function counting endlessly    
def endless_counter():
    num = 0
    while True:
        yield num
        num += 1


# can gram multiple instances from the same generator
# gen and gen2 both starts from 0
gen = endless_counter()
gen2 = endless_counter()
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen2))
print(next(gen2))

# also we can enumerare on generators
for i, num in enumerate(endless_counter()):
    if is_palindrome(num): print(num, end=' ')
    if i > 1000: print() ; break 


0
1
2
0
1
1 2 3 4 5 6 7 8 9 11 22 33 44 55 66 77 88 99 101 111 121 131 141 151 161 171 181 191 202 212 222 232 242 252 262 272 282 292 303 313 323 333 343 353 363 373 383 393 404 414 424 434 444 454 464 474 484 494 505 515 525 535 545 555 565 575 585 595 606 616 626 636 646 656 666 676 686 696 707 717 727 737 747 757 767 777 787 797 808 818 828 838 848 858 868 878 888 898 909 919 929 939 949 959 969 979 989 999 1001 


## Ahol "return" lehet, ott "yield" is

Az alábbi példában négy is van. Az ötödik next() pedig StopIteration exception-t dob

In [81]:
def multi_yield():
    out = 'először'
    yield out

    out = 'másodszor'
    yield out

    out = 'harmadszor'
    yield out

    out = 'negyedszer'
    yield out


gen = multi_yield()

print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))





először
másodszor
harmadszor
negyedszer


StopIteration: 

## A send() metódus használata  
A generátornak vissza is lehe tküldeni egy értéket, amit velhasználhat a további kiementek generálásához. Ennek érdekében a yield-et *expression*-ként használjuk, vagyis igényt tartunk a kimeneti értékére. Az alábbi pédlában az **inp** változó veszi fel a benemeti értéket, amint a generátor send() metódusával beküldünk. 

A send() csak akkor használható, ha már legylább egy next() volt rajta, vagyis aktiválva van. Viszont a send() hívása tartalmaz egy implicit next()-et is, szóval érték ebből is jön ki. 

In [166]:
import random
def random_yield():
    outlist = [
        'egyik',
        'másik',
        'harmadik',
        'negyedik',
    ]
    inp = None
    while True:
        out = random.choice(outlist)
        if inp != None:
            out = 'BUMP! "{}" {}'.format(inp,out)
        inp = ( yield out )
        #print('bejövő: {} '.format(inp))


randgen = random_yield()

print(next(randgen))
# print(next(randgen))
# print(next(randgen))
print(randgen.send('beküldött érték'))
print(next(randgen))
print(next(randgen))




egyik
BUMP! "beküldött érték" másik
egyik
egyik


## A throw() metódus

A RealPython cikkben szereplő példa a palindrom generátorban használja fel ezt. A send() által beállítható, hogy honnan kezdje keresni a következő értéket. 

Viszont ez olyat is csinál, hogy amikor úgy dönt, leállítja a generátort. A **throw()** metódust hívva rá lehet kényszeríteni a generátor függvényt, hogy hibát dojon. Itt épp egy **ValueError**-t

In [174]:
def infinite_palindromes():
    num = 0
    while True:
        if is_palindrome(num):
            i = (yield num)
            if i is not None:
                num = i
        num += 1

pal_gen = infinite_palindromes()
for i in pal_gen:
    print(i)
    digits = len(str(i))
    if digits == 5:
        pal_gen.throw(ValueError("We don't like large palindromes"))
        # pal_gen.close()
    pal_gen.send(10 ** (digits))

print('And here is the END') # idáig nem jutunk el



1
22
111
1111
10101


ValueError: We don't like large palindromes

## A close() metódus

Elvileg ezzel lehet szépen lezárni a generátort - rávenni egy StopIteration exception-re, amit a for ciklus pl szépen lekezel. 
Normál esetben legalábbis - véges iterátoroknál - lekezeli. Itt nem. A kód futása leáll a close()-ra is. **Miért?**

In [175]:
def infinite_palindromes():
    num = 0
    while True:
        if is_palindrome(num):
            i = (yield num)
            if i is not None:
                num = i
        num += 1

pal_gen = infinite_palindromes()
for i in pal_gen:
    print(i)
    digits = len(str(i))
    if digits == 5:
        # pal_gen.throw(ValueError("We don't like large palindromes"))
        pal_gen.close()
    pal_gen.send(10 ** (digits))

print('And here is the END') # idáig nem jutunk el ITT SEM



1
22
111
1111
10101


StopIteration: 

## Generátorok összefűzése data-pipeline -ná

A terv:
* Read every line of the file.
* Split each line into a list of values.
* Extract the column names.
* Use the column names and lists to create a dictionary.
* Filter out the rounds you aren’t interested in.
* Calculate the total and average values for the rounds you are interested in.

Az adatok a techchurch.csv-ben vannak

In [188]:

file_name = "techcrunch.csv"

# egy generator expresion a sorok kiolvasására
lines = (line for line in open(file_name))

# a kiolvasott sorokat listába bontjuk
list_line = (s.rstrip().split(",") for s in lines)

# felolvassuk az első sort - ebben lesznek a mező nevek
cols = next(list_line)

# átalakítjuk dictekké a listába tördelt sorokat úgy, hogy a
# *cols*-ból vesszük a kulcsértékeket. zip() és aztán dict()
company_dicts = (dict(zip(cols,data)) for data in list_line)

# ezzel a generátorral a company_dict-ek közül válogatunk feltételnek megfelelően 
# Mivel ez int-ekké konvertálva adja vissza az értékeket, lehet rajta sum()-ot használni
funding = (
    int(company_dict["raisedAmt"])
    for company_dict in company_dicts
    if company_dict["round"] == "a"
)

# Eddig nem történt szinte semmi (a fejléc felolvasásán kívül). 
# MOST a sum() által végigiterálunk az összes generátoron
# mert a sum()-nak egy Iterable-t adunk paraméternek és így azt végigrágja
total_series_a = sum(funding)



# végül ízlés szerint felhasználjuk az eredményt
print(f"Total series A fundraising: ${total_series_a}")


# eddig a generátorainkat kimerítettük, így ez a rész újrainicializálás nélkül hibát dob
# a sum()-os részt kikommentezve viszont működik
# a lényeg, hogy ebben az esetben for ciklussal iterálva meg is számoljuk a cégeket,
# hogy átlagot lehessen számolni.

# sum_a = 0 # ebbe gyűjtjük az összeget
# for i,a in enumerate(funding):
#     sum_a +=a

# i+=1 # mert nullával kezdtük a számolást
# print(f"Average funding is ${sum_a / i} in {i} companies")



Average funding is $7531867.469879518 in 581 companies
